In [9]:
from qiskit_ibm_provider import IBMProvider
from qiskit.providers import JobStatus

provider = IBMProvider(name='full')

In [3]:
%%bash

python examples/g14_on_ibm/run_ibm.py \
    --account=full \
    --shots=1024 \
    --circuit=data/g14_constrained_u3ry/g14_state.json

2023-09-06 13:11:00,682	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


Loaded account full


  0%|          | 0/15 [00:00<?, ?it/s]/home/jim/miniconda3/envs/adapt-gym/lib/python3.10/site-packages/qiskit_ibm_provider/ibm_backend.py:463: UserWarning: The backend ibm_sherbrooke is currently paused.
  warnings.warn(f"The backend {self.name} is currently paused.")


Submitted job cjsb5jik1qvd8tcm9kt0 to ibmq_jakarta
Submitted job cjsb5jmiel5ovfdojlqg to ibm_hanoi
Submitted job cjsb5jkiifjafcoacbb0 to ibmq_guadalupe
Submitted job cjsb5j8cal0hh8jges00 to ibm_perth
Submitted job cjsb5k7toe8ecf8p5lr0 to ibm_auckland
Submitted job cjsb5kciifjafcoacbv0 to ibmq_quito
Submitted job cjsb5kmiel5ovfdojmn0 to ibm_sherbrooke
Submitted job cjsb5lnp8blo0km91iog to ibm_cairo
Submitted job cjsb5lak1qvd8tcm9mh0 to ibmq_belem
Submitted job cjsb5leiel5ovfdojnh0 to ibmq_kolkata
Submitted job cjsb5mftoe8ecf8p5o10 to ibmq_mumbai
Submitted job cjsb5mocal0hh8jgeut0 to ibm_lagos
Submitted job cjsb5mvtoe8ecf8p5ob0 to ibmq_lima
Submitted job cjsb5mmiel5ovfdojoj0 to ibmq_manila
Submitted job cjsb5nnp8blo0km91kg0 to ibm_nairobi
